Group 3 - Comparing accuracies between Resnet and OpenCV regarding Human Faces

Metadata Exploratory Data Analysis

In [ ]:
pip install -r requirements.txt

In [ ]:
from pyspark.sql.functions import *

In [ ]:
%%capture
!pip install import_ipynb

# import the jupyter notebook importer

import import_ipynb
from data603 import SparkLauncher
from data603 import HDFS

In [ ]:
import import_ipynb
from data603 import SparkLauncher

# get a configuration object
conf = SparkLauncher.get_spark_conf()

# add a file to the configuration that will get copied to all the nodes on the cluster
conf.set('spark.yarn.dist.files', 'keras_data/resnet50_weights_tf_dim_ordering_tf_kernels.h5')

# Apache Arrow Config
#conf.set('spark.yarn.appMasterEnv.ARROW_PRE_0_15_IPC_FORMAT', '1')
#conf.set('spark.executorEnv.ARROW_PRE_0_15_IPC_FORMAT', '1')
#conf.set('spark.sql.execution.arrow.enabled', 'true')

# launch the cluster using the configuration
spark = SparkLauncher.get_spark_session(pack_venv = False, conf = conf)


Creating Spark Configuration
Setting Environment Variables
Creating Spark Session: bthota1_data603_spark_session


In [ ]:
#spark = SparkLauncher.get_spark_session(pack_venv = False)
hdfs = HDFS.get_hdfs()

In [ ]:
conf.set('spark.dynamicAllocation.minExecutors', '1')
conf.set('spark.dynamicAllocation.maxExecutors', '7')
conf.set('spark.executor.cores', '16')
conf.set('spark.executor.memory', '20g')

In [ ]:
image_files = hdfs.ls('/etl/google_open_image/images')
image_files

['/etl/google_open_image/images/test.avro',
 '/etl/google_open_image/images/train_0.avro',
 '/etl/google_open_image/images/train_1.avro',
 '/etl/google_open_image/images/train_2.avro',
 '/etl/google_open_image/images/train_3.avro',
 '/etl/google_open_image/images/train_4.avro',
 '/etl/google_open_image/images/train_5.avro',
 '/etl/google_open_image/images/train_6.avro',
 '/etl/google_open_image/images/train_7.avro',
 '/etl/google_open_image/images/train_8.avro',
 '/etl/google_open_image/images/train_9.avro',
 '/etl/google_open_image/images/train_a.avro',
 '/etl/google_open_image/images/train_b.avro',
 '/etl/google_open_image/images/train_c.avro',
 '/etl/google_open_image/images/train_d.avro',
 '/etl/google_open_image/images/train_e.avro',
 '/etl/google_open_image/images/train_f.avro',
 '/etl/google_open_image/images/validation.avro']

In [ ]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, StringType, IntegerType

df1=spark.read.csv('/data/google_open_image/metadata/class-descriptions-boxable.csv', 
                        schema = StructType([StructField("LabelName", StringType()), 
                                             StructField("LabelText", StringType())]) )

In [ ]:
df1.columns

['LabelName', 'LabelText']

In [ ]:
df1.limit(5)

DataFrame[LabelName: string, LabelText: string]

In [ ]:
face = df1.where("LabelText = 'Human face'")
face_label = face.toPandas()['LabelName'][0]
face_label

'/m/0dzct'

In [ ]:
df2=spark.read.csv('/data/google_open_image/labels/*.csv',header=True)

In [ ]:
df2.limit(5).toPandas()

,ImageID,Source,LabelName,Confidence
0,000026e7ee790996,verification,/m/0cgh4,0
1,000026e7ee790996,verification,/m/04hgtk,0
2,000026e7ee790996,verification,/m/0d5gx,0
3,000026e7ee790996,verification,/m/07j7r,1
4,000026e7ee790996,verification,/m/04rky,0


In [ ]:
face_ids = df2.where("LabelName = '/m/0dzct'").where("Confidence > 0.99")
face_ids.limit(5).toPandas()

,ImageID,Source,LabelName,Confidence
0,000132c20b84269b,verification,/m/0dzct,1
1,0002ab0af02e4a77,verification,/m/0dzct,1
2,000dbcc283d87f67,verification,/m/0dzct,1
3,0013bd1e79c796c0,verification,/m/0dzct,1
4,0019ac4eb659f57e,verification,/m/0dzct,1


In [ ]:
df3 = spark.read.parquet('/etl/google_open_image/images.parquet')

In [ ]:
df3.limit(5).toPandas()

,ImageID,Subset,Format,Data,OriginalURL,OriginalLandingURL,License,AuthorProfileURL,Author,Title,OriginalSize,OriginalMD5,Thumbnail300KURL,Rotation
0,0000d59fa570d973,train,JPG,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...",https://farm4.staticflickr.com/3165/2548414279...,https://www.flickr.com/photos/shkumbin/2548414279,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/shkumbin/,Shkumbin Saneja,James Blunt Tour 5,778226,nPqr5Ngx1hTYH381eq4fyQ==,https://c6.staticflickr.com/4/3165/2548414279_...,0.0
1,000213f8efef7523,train,JPG,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...",https://farm4.staticflickr.com/2384/5806700337...,https://www.flickr.com/photos/eddale/5806700337,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/eddale/,Ed Dale,DSCF1089,253430,VmHmzCMRyW/merIPvVwkwg==,https://c4.staticflickr.com/3/2384/5806700337_...,0.0
2,00031c22192e3389,train,JPG,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...",https://c2.staticflickr.com/4/3166/5722519230_...,https://www.flickr.com/photos/dchousegrooves/5...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/dchousegrooves/,Andy Grant,Caroline-birthday-party-apr-16-2011-048,383393,NpSKmVKWSM6BmQUGsY7UVw==,https://c6.staticflickr.com/4/3166/5722519230_...,None
3,000376d44b3be92e,train,JPG,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...",https://c7.staticflickr.com/4/3867/14856210396...,https://www.flickr.com/photos/29638108@N06/148...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/29638108@N06/,Jennifer C.,Jericho Park,2807676,OEL8PvC/vSnU0l4H3tWYKw==,https://c1.staticflickr.com/4/3867/14856210396...,0.0
4,00067defa682098f,train,JPG,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...",https://c8.staticflickr.com/3/2909/14561781027...,https://www.flickr.com/photos/lirneasia/145617...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/lirneasia/,lirneasia,IMG_5814,3648600,ARNiegszsRjsFUEgxlXQiw==,https://c2.staticflickr.com/3/2909/14561781027...,0.0


In [ ]:
df = df3.select(['ImageID','Data'])
df.limit(5).toPandas()

,ImageID,Data
0,0000d59fa570d973,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,..."
1,000213f8efef7523,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,..."
2,00031c22192e3389,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,..."
3,000376d44b3be92e,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,..."
4,00067defa682098f,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,..."


In [ ]:
face_data=face_ids.join(df,'ImageID', 'inner')

Combining the dataset for human faces and its coordinates

In [ ]:
bounding_boxes=spark.read.csv('/data/google_open_image/bboxes/*.csv',header=True)

In [ ]:
bounding_boxes.limit(5).toPandas()

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
0,000026e7ee790996,xclick,/m/07j7r,1,0.071875,0.1453125,0.20625,0.39166668,0,1,1,0,0
1,000026e7ee790996,xclick,/m/07j7r,1,0.4390625,0.571875,0.26458332,0.43541667,0,1,1,0,0
2,000026e7ee790996,xclick,/m/07j7r,1,0.66875,1,0,0.5520833,0,1,1,0,0
3,000062a39995e348,xclick,/m/015p6,1,0.20620842,0.849224,0.15463917,1,0,0,0,0,0
4,000062a39995e348,xclick,/m/05s2s,1,0.13747229,0.37694013,0,0.88365245,1,1,0,0,0


In [ ]:
# Changing the datatypes of the some columns
from pyspark.sql.types import DoubleType

bounding_boxes = bounding_boxes.withColumn("XMin", bounding_boxes['XMin'].cast(DoubleType()))
bounding_boxes = bounding_boxes.withColumn("XMax", bounding_boxes['XMax'].cast(DoubleType()))
bounding_boxes = bounding_boxes.withColumn("YMin", bounding_boxes['YMin'].cast(DoubleType()))
bounding_boxes = bounding_boxes.withColumn("YMax", bounding_boxes['YMax'].cast(DoubleType()))
bounding_boxes = bounding_boxes.withColumn("Confidence", bounding_boxes['Confidence'].cast(DoubleType()))

bounding_boxes.printSchema()

root
 |-- ImageID: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- LabelName: string (nullable = true)
 |-- Confidence: double (nullable = true)
 |-- XMin: double (nullable = true)
 |-- XMax: double (nullable = true)
 |-- YMin: double (nullable = true)
 |-- YMax: double (nullable = true)
 |-- IsOccluded: string (nullable = true)
 |-- IsTruncated: string (nullable = true)
 |-- IsGroupOf: string (nullable = true)
 |-- IsDepiction: string (nullable = true)
 |-- IsInside: string (nullable = true)



In [ ]:
face_datafile = spark.read.parquet('/user/bthota1/sample_project2.parquet')
face_datafile.limit(2).toPandas()

,ImageID,Source,LabelName,Confidence,Data
0,001094620459cb98,verification,/m/0dzct,1,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,..."
1,001996790a987b55,verification,/m/0dzct,1,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,..."


In [ ]:
bounding_boxes_facedata=face_datafile.join(bounding_boxes, 'ImageID', 'inner')
print(bounding_boxes_facedata.count())

1509693


In [ ]:
bounding_boxes_facedata.limit(5).toPandas()

,ImageID,Source,LabelName,Confidence,Data,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
0,001094620459cb98,verification,/m/0dzct,1,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...",activemil,/m/04yx4,1.0,0.035625,0.674375,0.200750,0.983114,0,1,0,0,0
1,001094620459cb98,verification,/m/0dzct,1,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...",xclick,/m/014sv8,1.0,0.458750,0.511250,0.576923,0.642589,1,0,0,0,0
2,001094620459cb98,verification,/m/0dzct,1,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...",xclick,/m/014sv8,1.0,0.561250,0.615000,0.555347,0.596623,0,0,0,0,0
3,001094620459cb98,verification,/m/0dzct,1,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...",xclick,/m/0283dt1,1.0,0.491250,0.593750,0.771107,0.912758,0,0,0,0,0
4,001094620459cb98,verification,/m/0dzct,1,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...",xclick,/m/02p0tk3,1.0,0.000000,0.999375,0.220450,0.999062,0,0,0,0,0


In [ ]:
cols = ['Source','LabelName','Confidence','IsOccluded','IsTruncated','IsGroupOf','IsDepiction','IsInside']
bounding_boxes_facedata = bounding_boxes_facedata.drop(*cols)

In [ ]:
bounding_boxes_facedata.limit(5).toPandas()

,ImageID,Data,XMin,XMax,YMin,YMax
0,001094620459cb98,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...",0.035625,0.674375,0.200750,0.983114
1,001094620459cb98,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...",0.458750,0.511250,0.576923,0.642589
2,001094620459cb98,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...",0.561250,0.615000,0.555347,0.596623
3,001094620459cb98,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...",0.491250,0.593750,0.771107,0.912758
4,001094620459cb98,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...",0.000000,0.999375,0.220450,0.999062


In [ ]:
bounding_boxes_facedata.printSchema()

root
 |-- ImageID: string (nullable = true)
 |-- Data: binary (nullable = true)
 |-- XMin: double (nullable = true)
 |-- XMax: double (nullable = true)
 |-- YMin: double (nullable = true)
 |-- YMax: double (nullable = true)



In [ ]:
image_face_100 = bounding_boxes_facedata.limit(100)
image_face_100.write.parquet("/user/bthota1/image_face.parquet")

In [ ]:
hdfs.ls('/user/bthota1/')

['/user/bthota1/.sparkStaging',
 '/user/bthota1/homework2-part1.parquet',
 '/user/bthota1/homework2_part1.parquet',
 '/user/bthota1/image_face.parquet',
 '/user/bthota1/image_face_no_data.parquet',
 '/user/bthota1/sample_project.parquet',
 '/user/bthota1/sample_project1.parquet',
 '/user/bthota1/sample_project2.parquet']

In [ ]:
faceimages_parquet=spark.read.parquet("/user/bthota1/image_face.parquet")
faceimages_parquet.count()

100

In [ ]:
conf.set('spark.dynamicAllocation.minExecutors', '1')

conf.set('spark.dynamicAllocation.maxExecutors', '7')

conf.set('spark.executor.cores', '16')

conf.set('spark.executor.memory', '20g')